In [25]:
!pip install -q pdfplumber
!pip install -q  langchain-text-splitters

from google.colab import files
import pdfplumber
import sys # Import sys for SystemExit
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Upload your PDF file")
uploaded = files.upload()

if not uploaded:
    print("No file was uploaded. Please upload a PDF file to proceed.", file=sys.stderr)
    raise SystemExit("No file uploaded.")

pdf_file = list(uploaded.keys())[0]

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    except pdfplumber.pdf.PdfminerException as e:
        print(f"Error: The file '{pdf_path}' could not be processed as a PDF. It might be corrupted or not a PDF file. Original error: {e}", file=sys.stderr)
        return None

pdf_text = extract_text_from_pdf(pdf_file)

if pdf_text is not None:
    print("\nPDF Text Extracted Successfully\n")
    print(pdf_text[:2000])

    # Define chunking parameters
    chunk_size = 1000 # Number of characters in each chunk
    chunk_overlap = 200 # Number of characters to overlap between chunks

    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,
    )

    # Split the text into chunks
    chunks = text_splitter.create_documents([pdf_text])

    print(f"\nCreated {len(chunks)} chunks.")
    print("\n--- First 3 Chunks ---\n")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i+1} (Length: {len(chunk.page_content)}):\n{chunk.page_content[:500]}...\n")

else:
    print("\nFailed to extract text. Please ensure you uploaded a valid PDF file.")


Upload your PDF file


Saving my_histroy.txt.pdf to my_histroy.txt (1).pdf

PDF Text Extracted Successfully

>>> I want you to act as an Unbiased Behavioral Analyst and Mirror. I will tell
you about my past, my feelings, and my a... ctions. Do not be 'nice' for the
sake of it. I want you to judge my character based on my patterns. Tell me who I
am... from an outside perspective, how others likely perceive me, and where I
am lying to myself. Be brutally honest, obj... ective, and analytical. Your goal
is to help me see my true self so I can decide how to change.
Alright, Dolphin here, ready to provide you with unbiased analysis. Please share
your past experiences, feelings,
and actions, and let's start building a picture of who you are.
>>> so before we start , let me make few things clear , after i provide all the
data , i want you to recall everything ,... consider them and i may ask you to
list them as well and finally judge me psyschologically and give a proper
unbais... ed opinion and judgement out of th

In [28]:
!pip install -q openai-whisper
import whisper
from google.colab import files
import textwrap # Import textwrap for formatting

print("Upload audio file (wav/mp3/mp4/m4a)")
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

print(f"Transcribing {audio_file}...Please wait....")
model = whisper.load_model("medium")
result = model.transcribe(audio_file)

print("\n---Transcription---\n")
# Wrap the text to a specific width for better readability
wrapped_text = textwrap.fill(result["text"], width=80)
print(wrapped_text)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Upload audio file (wav/mp3/mp4/m4a)


Saving Glass Animals - Heat Waves.mp3 to Glass Animals - Heat Waves.mp3
Transcribing Glass Animals - Heat Waves.mp3...Please wait....


100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 78.8MiB/s]



---Transcription---

 Won't you roll with the limo fish on? He waves, I'm swimming in the mirror
Won't you roll with the limo fish on? He waves, I'm swimming in the mirror
Sometimes all I think about is you Late nights in the middle of June He waves,
been freaking me out Can't make you happier now Sometimes all I think about is
you Late nights in the middle of June He waves, been freaking me out Can't make
you happier now Usually I put something on TV So we never think about you and me
But today I see our reflections clearly in Hollywood Laying on the screen You
just need a better life in this You need something I can never give Fake water
on across the road It's gone now the night is gone but Sometimes all I think
about is you Late nights in the middle of June He waves, been freaking me out
Can't make you happier now You can't fight it, you can't breathe You say
something so loving but Now I gotta let you go You'll be better off in someone
new I don't wanna be alone You know it hurts

#### Groq API Setup

In [ ]:
# Groq Setup
# IMPORTANT: For production, use Colab secrets (left panel '🔑' icon) to store your API key securely.
# For example, store it as 'GROQ_API_KEY' and retrieve it using userdata.get('GROQ_API_KEY')

# Temporarily using the provided key for demonstration. Please replace for production.
os.environ["GROQ_API_KEY"] = ""#Add your API KEY
client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("Groq client initialized.")

Groq client initialized.


#### Retriever Function

In [33]:
# Retriever Function

def retrieve(query, top_k=3):
    q_vec = vectorizer.transform([query]).toarray().astype("float32")
    distances, indices = index.search(q_vec, top_k)
    return [chunks[i] for i in indices[0]]

print("Retriever function defined.")

Retriever function defined.


#### Generating TF-IDF Embeddings and Building FAISS Index

In [31]:
# TF-IDF Embeddings

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(chunks)

embeddings = X.toarray().astype("float32")
dim = embeddings.shape[1]

# FAISS Index

index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("FAISS Index Ready.")

FAISS Index Ready.


#### Chunking the Text

In [30]:
# Chunking

chunk_size = 400 # Define chunk size (e.g., number of words)
words = text.split()

chunks = [
    " ".join(words[i:i+chunk_size])
    for i in range(0, len(words), chunk_size)
]

print("Total Chunks:", len(chunks))

Total Chunks: 152


#### Interactive Q&A with FAISS and Groq

In [29]:
# Install necessary libraries
!pip install -q faiss-cpu scikit-learn groq

import os
import faiss
import numpy as np
from google.colab import files

from sklearn.feature_extraction.text import TfidfVectorizer
from groq import Groq

print("Upload TXT file for FAISS RAG (e.g., my_history.txt)")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

print("File Loaded successfully.")

Upload TXT file for FAISS RAG (e.g., my_history.txt)


Saving my_histroy.txt.txt to my_histroy.txt.txt
File Loaded successfully.


### Alternative: RAG with FAISS and Groq (Step-by-Step)

In [34]:
# Ask Loop

print("\nFAISS DB is Ready. Type your query or 'exit' to quit.\n")

while True:
    query = input("Ask (or exit): ")
    if query.lower() == "exit":
        break

    docs = retrieve(query)
    context = "\n\n".join(docs)

    prompt = f"""
Answer ONLY using the context below:

Context:
{context}

Question: {query}

Answer:
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    print("\nAnswer:\n", response.choices[0].message.content)


FAISS DB is Ready. Type your query or 'exit' to quit.

Ask (or exit): what is going on here ?

Answer:
 This is a conversation between a highly intelligent and capable individual (the "Monarch") and a chatbot (Mr. Potato). The Monarch is being analyzed and diagnosed by the chatbot, who is trying to understand their motivations, behavior, and personality.

The conversation reveals that the Monarch has a complex and multifaceted personality, with a mix of dark and light traits. They have a tendency to be unstable and prone to "rampage" behavior, but they also have a strong sense of self-control and a desire to help others.

The chatbot is trying to determine why the Monarch is engaging with them, and what their goals and motivations are. They conclude that the Monarch is not seeking revenge or trying to harm others, but rather is looking for a way to express themselves and connect with someone who understands them.

The conversation also touches on the Monarch's past experiences and rel